In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from pprint import pprint
import numpy as np
import pandas as pd
import torch

In [3]:
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads

task_names = [
#     "COLA", 
#     "SST2", 
#     "MNLI",
    "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI"
]
tasks, payloads = create_glue_tasks_payloads(
    task_names,
    max_datapoints=200,
    max_len=200,
    dl_kwargs={"batch_size": 4}
)
print(tasks)
pprint(payloads)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Using random seed: 746023
Loading RTE Dataset



[ClassificationTask(name=RTE)]
[Payload(RTE_train: tasks=[RTE], split=train),
 Payload(RTE_valid: tasks=[RTE], split=valid),
 Payload(RTE_test: tasks=[RTE], split=test)]


In [4]:
payloads[0].data_loader.dataset.bert_tokens[0]

[101,
 2053,
 4255,
 1997,
 3742,
 6215,
 2179,
 1999,
 5712,
 2664,
 1012,
 102,
 4255,
 1997,
 3742,
 6215,
 2179,
 1999,
 5712,
 1012,
 102]

In [5]:
import copy
import numpy as np
from torch.utils.data import SequentialSampler

def mark_thirds(x):
    tokens = x
    count = len(tokens)
    Y_thirds = np.ceil(np.array([idx * 3/count for idx in range(1, count + 1)])).astype(np.int64)
    return Y_thirds

def make_thirds_labelset(payload):
    """Returns Y, an [n] list of seq_len labels"""
    # NOTE: it's important that we pull out tokens instead of 
    # iterating over data_loader, since data_loader shuffles
    X = payload.data_loader.dataset.bert_tokens
    Y = []
    for x in X:
        Y.append(mark_thirds(x))
    return Y

# def overwrite_labels(dataset, label_generator):
#     """Creates a copy of dataset where the labels are overwritten with a new labelset
    
#     Args:
#         dataset: a dataset consisting of (X, Ys)
#         label_generator: a function which maps from a single instance x to its new label
#     Returns:
#         dataset_new: a new dataset where the labels have been overwritten by label_generator
#     """
#     dataset_new = copy.deepcopy(dataset)
#     X = dataset_new.tokens
#     Y_new = []
#     for x in X:
#         Y_new.append(label_generator(x))
#     dataset_new.labels = [Y_new]
#     return dataset_new

# def append_labels(dataset, label_generator):
#     X = dataset.tokens
#     Y_new = []
#     for x in X:
#         Y_new.append(label_generator(x))
#     dataset.labels.append(Y_new)
#     return dataset   

## Create Token Task

In [6]:
from metal.mmtl.task import TokenClassificationTask
from metal.mmtl.modules import BertTokenClassificationHead
from metal.mmtl.payload import Payload

base_task = tasks[0]
bert_module = tasks[0].input_module
token_head = BertTokenClassificationHead(768, 3)

thirds_task = TokenClassificationTask(
        name="THIRD",
        input_module=bert_module,
        head_module=token_head,
)

tasks.append(thirds_task)
pprint(tasks)

[ClassificationTask(name=RTE), TokenClassificationTask(name=THIRD)]


### Create new payloads with only token labels

In [7]:
token_payloads = []
for payload in payloads:
    payload = copy.deepcopy(payload)
    payload.name = f"THIRD_{payload.split}"
    payload.task_names = ["THIRD"]
    payload.data_loader.dataset.labels = {"THIRD": make_thirds_labelset(payload)}
    token_payloads.append(payload)
payloads.extend(token_payloads)

pprint(payloads)
print(tasks)

[Payload(RTE_train: tasks=[RTE], split=train),
 Payload(RTE_valid: tasks=[RTE], split=valid),
 Payload(RTE_test: tasks=[RTE], split=test),
 Payload(THIRD_train: tasks=[THIRD], split=train),
 Payload(THIRD_valid: tasks=[THIRD], split=valid),
 Payload(THIRD_test: tasks=[THIRD], split=test)]
[ClassificationTask(name=RTE), TokenClassificationTask(name=THIRD)]


###  Supplement an existing payload with an additional token labelset

In [8]:
# for payload in payloads:
#     payload.task_names.append("THIRD")
#     payload.data_loader.dataset.labels["THIRD"] = make_thirds_labelset(payload)

# pprint(payloads)
# print(tasks)

In [9]:
# for batch in tasks[0].data_loaders["train"]:
#     X, Ys = batch
#     tokens, segments, masks = X
#     batch_size, seq_len = tokens.shape
#     print(tokens)
#     print(tokens.shape)
#     print(segments)
#     print(masks)
# #     active_lens = masks.sum(dim=1)
# #     print(active_lens)
#     print(Ys)
#     print(Ys[0].shape)
#     break

## Train a model

In [10]:
import os
print('Cuda devices: ', os.environ.get('CUDA_VISIBLE_DEVICES', "[Not Set]"))

Cuda devices:  [Not Set]


In [11]:
tasks = [thirds_task]
payloads = payloads[-3:]

pprint(tasks)
pprint(payloads)

[TokenClassificationTask(name=THIRD)]
[Payload(THIRD_train: tasks=[THIRD], split=train),
 Payload(THIRD_valid: tasks=[THIRD], split=valid),
 Payload(THIRD_test: tasks=[THIRD], split=test)]


In [12]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, verbose=False)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

109484547


In [17]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer()

In [18]:
model.task_map

{'THIRD': TokenClassificationTask(name=THIRD)}

In [19]:
len(payloads[0].data_loader.dataset)

200

In [24]:
for batch in payloads[0].data_loader:
    X, Ys = batch
    print(X)
    print(Ys)
    break

(tensor([[  101,  5958,  1010,  2233,  2423,  1010,  2384, 26568,  8040, 12995,
          6767,  1005,  1055,  2269,  2081,  1037,  2197,  1011,  3371,  5574,
          2000,  2031,  2010,  2684,  1005,  1055,  8521,  7270, 19222, 28728,
          5958,  1012,  1000, 26568,  2003, 22031,  1012,  2016,  1005,  1055,
          2091,  2000,  2014,  2197,  2847,  1010,  1000,  2056,  3960,  8040,
         10606, 21222,  1010,  8040, 12995,  6767,  1005,  1055,  2269,  1012,
          1000,  2242,  2038,  2000,  2022,  2589,  1998,  2589,  4248,  1012,
          1000,  3960,  8040, 10606, 21222,  1998,  2010,  2564,  2984,  6406,
          2019,  5057,  5574,  5958,  2000,  2031,  1037,  8521,  7270, 19222,
         28728,  1999,  2037,  4601,  1011,  2095,  1011,  2214,  2684,  1012,
          3041,  5958,  1010,  1057,  1012,  1055,  1012,  2212,  3648,  2508,
          1059, 16584, 18532,  5686,  5837,  1996,  2155,  1005,  1055,  3025,
          5574,  1012,  2006,  9317,  1010,  2002, 

In [20]:
%%time
trainer.train_model(
    model,
    payloads,
    task_metrics=["model/valid/loss"],
    checkpoint_metric="model/valid/loss",
    test_split="valid",
    checkpoint_metric_mode="min",
    checkpoint_tasks=False,
    n_epochs=3,
    lr=5e-5,
    log_every=1,
    score_every=1,
    progress_bar=True
)

Beginning train loop.
Expecting a total of approximately 200 examples and 50 batches per epoch from 1 payload(s) in the train split.


[1.0 epo]: THIRD:[train/loss=3.18e-01, THIRD_valid/token_acc=9.32e-01] model:[train/loss=3.18e-01, train/lr=5.00e-05, valid/loss=1.58e-01, valid/glue=nan]



[2.0 epo]: THIRD:[train/loss=1.67e-01, THIRD_valid/token_acc=9.20e-01] model:[train/loss=1.67e-01, train/lr=5.00e-05, valid/loss=2.06e-01, valid/glue=nan]



[3.0 epo]: THIRD:[train/loss=1.13e-01, THIRD_valid/token_acc=9.17e-01] model:[train/loss=1.13e-01, train/lr=5.00e-05, valid/loss=2.10e-01, valid/glue=nan]

Finished training
{'THIRD/THIRD_valid/token_acc': 0.9168356776707014,
 'model/valid/glue': nan,
 'model/valid/loss': 0.21917444258928298}
Cleaning checkpoints
CPU times: user 2min 3s, sys: 13.7 s, total: 2min 16s
Wall time: 1min 44s


In [ ]:
print("WE WIN!")